In [7]:
import json

# Demonstration of how to load a file that contains secrets without accidentally leaking those secrets
with open("C:\\Users\\Owner\\OneDrive\\Documents\\Automate Boring Stuff\\secrets.json") as f:
    data = json.load(f)

    # If you want your data to be secure, don't print this variable out!
    # Juputer will retain a cached version of any printed data and it can be
    # accidentally committed to version control.
    secret_key = data["mongodb"]

# We can safely print the length of the secret key. That won't leak any sensitive information.
print(f'My secret key is {len(secret_key)} characters in length')

My secret key is 68 characters in length


In [8]:
import pymongo  
import certifi

# Connect to the database using known good certificates
client = pymongo.MongoClient(secret_key, tlsCAFile = certifi.where())
print(f"Using MongoDB version {client.server_info()['version']}.")

# Check what databases exist on this server 
all_databases = client.list_database_names()
print(f'This MongoDB server has the databases {all_databases})')

# If we know the correct database to talk to, we connect like this:
my_database = client['Database1']

# Here is the list of collections within my database
all_collections = my_database.list_collection_names()
print(f'This database has the collections {all_collections}')

Using MongoDB version 7.0.15.
This MongoDB server has the databases ['Database1', 'sample_mflix', 'admin', 'local'])
This database has the collections ['IMDB', 'Metacritic']


In [9]:
import pandas as pd 
# Retrieve all records from a collection - this can be a large amount of data!
cursor = my_database['IMDB'].find()

# Convert this information into a Pandas dataframe
imdb = pd.DataFrame(cursor)

# Make sure we've read the information correctly
# imdb.head()

# Transform and standardize the columns
imdb.release_date = pd.to_datetime(imdb.release_date, errors = 'coerce')
imdb.budget = pd.to_numeric(imdb.budget, errors = 'coerce')
imdb.runtime = pd.to_numeric(imdb.runtime, errors = 'coerce')

# Filter for the year 2008
imdb_2003 = imdb[imdb['release_date'].dt.year == 2003]

# Run a report sorted by budget - lowest to highest
imdb_2003.sort_values(by=['budget']).head()

,_id,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company
9505,6714d970a5f947788080c422,390538,Tarnation,88.0,7.1,6575,Unrated,2003-10-19,218.0,12740.0,NaN,"Documentary, Biography","Renee Leblanc, Jonathan Caouette, Adolph Davis...",Jonathan Caouette,"Vanessa Arteaga, Jason Banker, Jonathan Caouet...","Tarnation Films, Wellspring Media"
9751,6714d970a5f947788080c518,400172,Anus Magillicutty,70.0,2.3,3765,Not Rated,2003-04-15,5000.0,NaN,NaN,"Comedy, Romance","Lloyd Garner III, Paige Abbott, Joe Hall, Budd...",Morey Fineburgh,"Lloyd Garner III, Buddy Golden, Moishe Goldfarb","Brothers of the Order, Corporations Unlimited"
9383,6714d970a5f947788080c3a8,425027,Flywheel,120.0,6.6,4423,Not Rated,2003-04-09,20000.0,NaN,NaN,Drama,"Rosetta Harris Armstrong, Lisa Arnold, Blake B...",Alex Kendrick,"Lisa Arnold, Barry Carr, Michael Catt, Tracy G...","Carmel Entertainment, Provident Films, Sherwoo..."
9434,6714d970a5f947788080c3db,368913,Osama,83.0,7.3,9112,PG-13,2003-05-20,46000.0,NaN,NaN,"Drama, War","Marina Golbahari, Khwaja Nader, Mohammad Arif ...",Siddiq Barmak,"Siddiq Barmak, Julia Fraser, Julie LeBrocquy, ...","Barmak Film, LeBrocquy Fraser Productions, NHK..."
9223,6714d970a5f947788080c308,374102,Open Water,79.0,5.8,53609,R,2003-10-26,120000.0,1100943.0,54667954.0,"Adventure, Drama, Horror, Thriller","Blanchard Ryan, Daniel Travis, Saul Stein, Mic...",Chris Kentis,"Estelle Lau, Laura Lau, Ruben Samuel Sachs",Plunge Pictures LLC


In [10]:
# Sort by budget to find outliers
sorted_budget = imdb_2003.sort_values(by='budget')

# Calculate the IQR for budget
Q1_budget = sorted_budget['budget'].quantile(0.25)
Q3_budget = sorted_budget['budget'].quantile(0.75)
IQR_budget = Q3_budget - Q1_budget

# Define outlier threshold for budget
lower_bound_budget = Q1_budget - 1.5 * IQR_budget
upper_bound_budget = Q3_budget + 1.5 * IQR_budget

# Identify budget outliers
budget_outliers = sorted_budget[(sorted_budget['budget'] < lower_bound_budget) | (sorted_budget['budget'] > upper_bound_budget)]
print('Top Budget Outliers:')
print(budget_outliers[['title', 'budget']]) 

Top Budget Outliers:
                                                  title       budget
9205                                The Haunted Mansion   90000000.0
9117      The Lord of the Rings: The Return of the King   94000000.0
9124                                       Finding Nemo   94000000.0
9210         Lara Croft Tomb Raider: The Cradle of Life   95000000.0
9298                                            Control  100000000.0
9135                                          Peter Pan  100000000.0
9629                                    Pudhiya Geethai  100000000.0
9142                                 The Cat in the Hat  109000000.0
9133                                   X2: X-Men United  110000000.0
9560                                             Vishnu  120000000.0
9386                                      Chalte Chalte  120000000.0
9187                    Charlie's Angels: Full Throttle  120000000.0
9194                                       Brother Bear  128000000.0
9150         

In [11]:
# Sort by runtime to find outliers
sorted_runtime = imdb_2003.sort_values(by='runtime')

# Calculate the IQR for budget
Q1_runtime = sorted_runtime['runtime'].quantile(0.25)
Q3_runtime = sorted_runtime['runtime'].quantile(0.75)
IQR_runtime = Q3_runtime - Q1_runtime

# Define outlier threshold for runtime
lower_bound_runtime = Q1_runtime - 1.5 * IQR_runtime
upper_bound_runtime = Q3_runtime + 1.5 * IQR_runtime

# Identify runtime outliers
runtime_outliers = sorted_runtime[(sorted_runtime['runtime'] < lower_bound_runtime) | (sorted_runtime['runtime'] > upper_bound_runtime)]
print('Top Runtime Outliers:')
print(budget_outliers[['title', 'runtime']]) 

Top Runtime Outliers:
                                                  title  runtime
9205                                The Haunted Mansion     99.0
9117      The Lord of the Rings: The Return of the King    201.0
9124                                       Finding Nemo    100.0
9210         Lara Croft Tomb Raider: The Cradle of Life    117.0
9298                                            Control    111.0
9135                                          Peter Pan    113.0
9629                                    Pudhiya Geethai    140.0
9142                                 The Cat in the Hat     82.0
9133                                   X2: X-Men United    134.0
9560                                             Vishnu    180.0
9386                                      Chalte Chalte    167.0
9187                    Charlie's Angels: Full Throttle    106.0
9194                                       Brother Bear     85.0
9150                                        Bad Boys II    147.0
912

```
The movies seen above as budegt or runtime outliers probably differ from typical production standards

For budget outliers, high budget films could have large scale productions, alot of special effects, or high profile actors, while low budgets might be experimental films with limited resources.

For runtime outliers, long films might be in storytelling formats, while short runtimes could be short films or documentaries. 
```